In [ ]:
#API
import requests

#ETL
import pandas as pd
from pandas import json_normalize
import time
import json
import numpy as np
from datetime import datetime, timezone
from pathlib import Path

#DB
# import psycopg2
# from psycopg2.extras import Json
from sqlalchemy import create_engine, text
# from supabase import create_client, Client
from sqlalchemy.exc import SQLAlchemyError

#Other
import os
from dotenv import load_dotenv

In [ ]:
load_dotenv()
api_key = os.getenv('api_key')
db_password = os.getenv('db_password')
db_user = os.getenv('db_user')
db_host = os.getenv('db_host')
db_port = os.getenv('db_port')
db_name = os.getenv('db_name')

In [ ]:
headers = {"Authorization": f"Bearer {api_key}"}

In [ ]:
# All league identifiers

#S54
advanced_id = '786da1fa-a9b5-4d17-b40e-416588272db4'
main_id = 'bd7957b3-1114-435f-af43-22645a508cc0'
intermediate_id ='4410aa73-9584-4500-a7ca-1504ed6da707'
entry_id ='e61be909-1727-4de3-8201-419d89893401'
open10_id ='f62c4c75-f620-40a2-bef8-0be2cb85d2d3'
open9_id = '0ce757ef-9bad-4afa-bbce-58d647824222'
open1_8_id ='3cd81a7e-ed93-43ba-be08-2a1ee7ad7237'

region= '55a91ca4-fa67-48b1-aed7-ff635fecc3ed'

championship_ids = ['786da1fa-a9b5-4d17-b40e-416588272db4',
                    'bd7957b3-1114-435f-af43-22645a508cc0',
                    '4410aa73-9584-4500-a7ca-1504ed6da707',
                    'e61be909-1727-4de3-8201-419d89893401',
                    'f62c4c75-f620-40a2-bef8-0be2cb85d2d3',
                    '0ce757ef-9bad-4afa-bbce-58d647824222',
                    '3cd81a7e-ed93-43ba-be08-2a1ee7ad7237']

## Call API pour récupérer les données

### Détail des leagues

In [ ]:
## api call not working for some reason

### List of match played per championships

In [ ]:
def fetch_and_save_all_matches(championship_ids, headers, filename="all_matches.json", limit=100, delay=0.3):

    def get_all_matches_for_champ(championship_id):
        offset = 0
        champ_matches = []
        while True:
            url = f"https://open.faceit.com/data/v4/championships/{championship_id}/matches?region={region}&type=all&limit={limit}&offset={offset}"
            response = requests.get(url, headers=headers)

            if response.status_code != 200:
                print(f"Error for {championship_id} at offset {offset}: Status Code {response.status_code} - {response.text}")
                break

            data = response.json()
            items = data.get("items", [])

            if not items:
                print(f"No more matches found for championship {championship_id} at offset {offset}")
                break

            champ_matches.extend(items)
            print(f"Fetched {len(items)} matches for championship {championship_id} at offset {offset}")

            offset += limit
            time.sleep(delay)

        return champ_matches
    
    all_matches= []

    for champ_id in championship_ids:
        print(f"Fetching matches for championship: {champ_id}")
        matches = get_all_matches_for_champ(champ_id)
        print(f"Total matches found for championship {champ_id}: {len(matches)}")
        all_matches.extend(matches)

    with open(filename, "w") as f:
        json.dump(all_matches, f, indent=2)

    print(f"Total matches collected across all championships: {len(all_matches)}")

    return all_matches

In [ ]:
# all_matches = fetch_and_save_all_matches(championship_ids, headers,filename="all_up_matches.json", limit=100, delay=0.3)

with open("all_past_matches.json", "r", encoding="utf-8") as f:
    all_matches = json.load(f)

#### upcoming matches


In [ ]:

def fetch_and_save_upcoming_matches(championship_ids, headers, filename="all_up_matches.json", limit=100, delay=0.3):

    def get_upcoming_matches_for_champ(championship_id):
        offset = 0
        champ_matches = []
        while True:
            url = f"https://open.faceit.com/data/v4/championships/{championship_id}/matches?region={region}&type=upcoming&limit={limit}&offset={offset}"
            response = requests.get(url, headers=headers)

            if response.status_code != 200:
                print(f"Error for {championship_id} at offset {offset}: Status Code {response.status_code} - {response.text}")
                break

            data = response.json()
            items = data.get("items", [])

            if not items:
                print(f"No more matches found for championship {championship_id} at offset {offset}")
                break

            champ_matches.extend(items)
            print(f"Fetched {len(items)} matches for championship {championship_id} at offset {offset}")

            offset += limit
            time.sleep(delay)

        return champ_matches
    
    all_up_matches= []

    for champ_id in championship_ids:
        print(f"Fetching matches for championship: {champ_id}")
        matches = get_upcoming_matches_for_champ(champ_id)
        print(f"Total matches found for championship {champ_id}: {len(matches)}")
        all_up_matches.extend(matches)

    with open(filename, "w") as f:
        json.dump(all_up_matches, f, indent=2)

    print(f"Total matches collected across all championships: {len(all_up_matches)}")

    return all_up_matches

In [ ]:
# all_up_matches = fetch_and_save_upcoming_matches(championship_ids, headers,filename="all_up_matches.json", limit=100, delay=0.3)

with open("all_up_matches.json", "r", encoding="utf-8") as f:
    all_up_matches = json.load(f)

#### Past matches

In [ ]:

def fetch_and_save_past_matches(championship_ids, headers, filename="all_past_matches.json", limit=100, delay=0.3):

    def get_past_matches_for_champ(championship_id):
        offset = 0
        champ_matches = []
        while True:
            url = f"https://open.faceit.com/data/v4/championships/{championship_id}/matches?region={region}&type=past&limit={limit}&offset={offset}"
            response = requests.get(url, headers=headers)

            if response.status_code != 200:
                print(f"Error for {championship_id} at offset {offset}: Status Code {response.status_code} - {response.text}")
                break

            data = response.json()
            items = data.get("items", [])

            if not items:
                print(f"No more matches found for championship {championship_id} at offset {offset}")
                break

            champ_matches.extend(items)
            print(f"Fetched {len(items)} matches for championship {championship_id} at offset {offset}")

            offset += limit
            time.sleep(delay)

        return champ_matches
    
    all_past_matches= []

    for champ_id in championship_ids:
        print(f"Fetching matches for championship: {champ_id}")
        matches = get_past_matches_for_champ(champ_id)
        print(f"Total matches found for championship {champ_id}: {len(matches)}")
        all_past_matches.extend(matches)

    with open(filename, "w") as f:
        json.dump(all_past_matches, f, indent=2)

    print(f"Total matches collected across all championships: {len(all_past_matches)}")

    return all_past_matches

In [ ]:
#Using API or local  cached file

#all_past_matches = fetch_and_save_past_matches(championship_ids, headers,filename="all_matches.json", limit=100, delay=0.3)

with open("all_past_matches.json", "r", encoding="utf-8") as f:
    all_past_matches = json.load(f)


In [ ]:
match_list = [match['match_id'] for match in all_matches]

### Details of each match listed

In [ ]:
def fetch_match_stats(match_list, headers, stats_file="match_stats.json", no_stats_file="no_stats.json", delay=0.2):
    stats_path = Path(stats_file)
    no_stats_path = Path(no_stats_file)

    # 1. Charger les matchs avec stats existants
    if stats_path.exists():
        with open(stats_path, "r") as f:
            all_stats = json.load(f)
    else:
        all_stats = {}

    # 2. Charger matchs sans stats
    if no_stats_path.exists():
        with open(no_stats_path, "r") as f:
            no_stats = set(json.load(f))
    else:
        no_stats = set()

    # 3. Calculer les matchs à récupérer
    match_set = set(match_list)
    already_fetched = set(all_stats.keys())
    to_skip = already_fetched.union(no_stats)
    to_fetch = list(match_set - to_skip)

    print(f"🔍 {len(to_fetch)} nouveaux matchs à récupérer parmi {len(match_list)} total.\n")

    # 4. Boucle sur matchs à récupérer
    for i, match_id in enumerate(to_fetch, start=1):
        url = f"https://open.faceit.com/data/v4/matches/{match_id}/stats"

        try:
            response = requests.get(url, headers=headers)
            if response.status_code == 404:
                print(f"[{i}/{len(to_fetch)}] {match_id} → Pas de stats (404) ❌")
                no_stats.add(match_id)
                continue

            response.raise_for_status()
            match_data = response.json()

            all_stats[match_id] = match_data
            print(f"[{i}/{len(to_fetch)}] {match_id} récupéré ✅")

            time.sleep(delay)

        except requests.exceptions.RequestException as e:
            print(f"[{i}/{len(to_fetch)}] Erreur pour {match_id} ❌ : {e}")
            continue

    # 5. Sauvegarde finale
    with open(stats_path, "w") as f:
        json.dump(all_stats, f, indent=2)

    with open(no_stats_path, "w") as f:
        json.dump(list(no_stats), f, indent=2)

    print(f"\n✅ Terminé ! Total stockés : {len(all_stats)} ✅ | Sans stats : {len(no_stats)} ❌")


    return all_stats, no_stats


In [ ]:
def fetch_match_stats(
    match_list, 
    headers, 
    stats_file="match_stats.json", 
    no_stats_file="no_stats.json", 
    delay=0.2,
    test_mode=False
):
    # If test_mode is ON → redirect to test files
    if test_mode:
        stats_file = f"test_{stats_file}"
        no_stats_file = f"test_{no_stats_file}"

    stats_path = Path(stats_file)
    no_stats_path = Path(no_stats_file)

    # 1. Charger les matchs avec stats existants
    if stats_path.exists():
        with open(stats_path, "r") as f:
            all_stats = json.load(f)
    else:
        all_stats = {}

    # 2. Charger matchs sans stats
    if no_stats_path.exists():
        with open(no_stats_path, "r") as f:
            no_stats = set(json.load(f))
    else:
        no_stats = set()

    # 3. Calculer les matchs à récupérer
    match_set = set(match_list)
    already_fetched = set(all_stats.keys())
    to_skip = already_fetched.union(no_stats)
    to_fetch = list(match_set - to_skip)

    print(f"🔍 {len(to_fetch)} nouveaux matchs à récupérer parmi {len(match_list)} total.\n")

    # 4. Boucle sur matchs à récupérer
    for i, match_id in enumerate(to_fetch, start=1):
        url = f"https://open.faceit.com/data/v4/matches/{match_id}/stats"

        try:
            response = requests.get(url, headers=headers)
            if response.status_code == 404:
                print(f"[{i}/{len(to_fetch)}] {match_id} → Pas de stats (404) ❌")
                no_stats.add(match_id)
                continue

            response.raise_for_status()
            match_data = response.json()

            all_stats[match_id] = match_data
            print(f"[{i}/{len(to_fetch)}] {match_id} récupéré ✅")

            time.sleep(delay)

        except requests.exceptions.RequestException as e:
            print(f"[{i}/{len(to_fetch)}] Erreur pour {match_id} ❌ : {e}")
            continue

    # 5. Sauvegarde finale
    with open(stats_path, "w") as f:
        json.dump(all_stats, f, indent=2)

    with open(no_stats_path, "w") as f:
        json.dump(list(no_stats), f, indent=2)

    print(f"\n✅ Terminé ! Total stockés : {len(all_stats)} ✅ | Sans stats : {len(no_stats)} ❌")

    return all_stats, no_stats


In [ ]:
#using captain matches list (NEW)
# all_stats, no_stats = fetch_match_stats(
#     match_list=filtered_captain_matches, 
#     headers=headers, 
#     test_mode=True
# )

#using championship matches (OLD)
# all_stats, no_stats = fetch_match_stats(match_list, headers, stats_file="match_stats.json", no_stats_file="no_stats.json", delay=0.2)

with open("test_match_stats.json", "r", encoding="utf-8") as f:
    all_stats = json.load(f)

In [ ]:
team_ids = set()  # set pour éviter les doublons

for match_id, match_data in all_stats.items():
    rounds = match_data.get('rounds', [])
    for round_data in rounds:
        teams = round_data.get('teams', [])
        for team in teams:
            team_id = team.get('team_id')
            if team_id:
                team_ids.add(team_id)

team_ids = list(team_ids)

### Details of each team involved in matches related to championships

In [ ]:
def fetch_team_details(team_ids, headers, team_file="team_details.json", delay=0.3):
    team_path = Path(team_file)

    # Charger les données existantes si présentes
    if team_path.exists():
        with open(team_path, "r") as f:
            all_teams = json.load(f)
    else:
        all_teams = {}

    # Liste des IDs déjà récupérés
    already_fetched = set(all_teams.keys())

    # Filtrer les team_ids à récupérer
    to_fetch = [tid for tid in team_ids if tid not in already_fetched]

    print(f"🔍 {len(to_fetch)} nouvelles équipes à récupérer parmi {len(team_ids)} au total.\n")

    for i, team_id in enumerate(to_fetch, start=1):
        url = f"https://open.faceit.com/data/v4/teams/{team_id}"

        try:
            response = requests.get(url, headers=headers)
            if response.status_code == 404:
                print(f"[{i}/{len(to_fetch)}] {team_id} → Équipe introuvable (404) ❌")
                all_teams[team_id] = None  # Ou gérer comme tu veux
                continue

            response.raise_for_status()
            team_data = response.json()

            all_teams[team_id] = team_data
            print(f"[{i}/{len(to_fetch)}] {team_id} récupérée ✅")

            time.sleep(delay)

        except requests.exceptions.RequestException as e:
            print(f"[{i}/{len(to_fetch)}] Erreur pour {team_id} ❌ : {e}")
            continue

    # Sauvegarde finale
    with open(team_path, "w") as f:
        json.dump(all_teams, f, indent=2)

    print(f"\n✅ Terminé ! Total équipes stockées : {len(all_teams)}")

    return all_teams


In [ ]:
#Using API or local  cached file

# all_teams= fetch_team_details(team_ids, headers)

with open("team_details.json", "r", encoding="utf-8") as f:
    all_teams = json.load(f)

### Match list from captains of each team to overrun API limit 

In [ ]:
captains_list = []

for team_id, team_data in all_teams.items():
        row = {
            'leader': team_data.get('leader')
        }
        captains_list.append(row)

# Create DataFrame
captains_df = pd.DataFrame(captains_list)


In [ ]:
captains_ids = captains_df['leader'].tolist()

In [ ]:
## LAST PULL DONE FROM 01 JULY TO 20 AUGUST
start_season_unix = '1751328000'

In [ ]:
def fetch_all_captain_matches(captains_ids, start_season_unix, api_key, limit=100):
    headers = {"Authorization": f"Bearer {api_key}"}
    all_captain_matches = {}

    for player_id in captains_ids:
        match_info = {} 
        offset = 0

        while True:
            url = f"https://open.faceit.com/data/v4/players/{player_id}/history?game=cs2&from={start_season_unix}&limit={limit}&offset={offset}"
            response = requests.get(url, headers=headers)

            if response.status_code != 200:
                print(f"Erreur pour {player_id}: {response.status_code}")
                break

            data = response.json()
            matches = data.get("items", [])
            print(f"{player_id} offset {offset}: retrieved {len(matches)} matches")  # debug

            if not matches:
                break

            for match in matches:
                match_id = match["match_id"]
                competition_id = match.get("competition_id")
                started_at = match.get("started_at")

                faction1_team_id = match.get("teams", {}).get("faction1", {}).get("team_id")
                faction2_team_id = match.get("teams", {}).get("faction2", {}).get("team_id")
                winner = match.get("results", {}).get("winner")

                match_info[match_id] = {
                    "competition_id": competition_id,
                    "started_at": started_at,
                    "faction1_team_id": faction1_team_id,
                    "faction2_team_id": faction2_team_id,
                    "winner": winner
                }
            # Pagination
            offset += limit

            # Stop if fewer matches returned than limit
            if len(matches) < limit:
                break

            time.sleep(0.5)  # rate limit

        all_captain_matches[player_id] = match_info
        print(f"{player_id}: total {len(match_info)} matches retrieved.")

    # Write all matches to JSON
    with open("all_captain_matches.json", "w") as f:
        json.dump(all_captain_matches, f, indent=2)

    return all_captain_matches

In [ ]:
# all_captain_matches = fetch_all_captain_matches(captains_ids, start_season_unix, api_key, limit=100)
with open("all_captain_matches.json", "r", encoding="utf-8") as f:
    all_captain_matches = json.load(f)

In [ ]:
rows = []
for player_id, matches in all_captain_matches.items():
    for match_id, match_info in matches.items():
        rows.append({
            "match_id": match_id,
            "competition_id": match_info.get("competition_id"),
            "match_day": match_info.get("started_at"),
            "faction1_team_id" : match_info.get("faction1_team_id"),
            "faction2_team_id": match_info.get("faction2_team_id"),
            "winner": match_info.get("winner"),
        })

# Convert to DataFrame
all_captain_matches_df = pd.DataFrame(rows)

# Optional: convert started_at to datetime
all_captain_matches_df["match_day"] = pd.to_datetime(all_captain_matches_df["match_day"], unit="s")

In [ ]:
filtered_captain_matches_df = all_captain_matches_df[all_captain_matches_df['competition_id'].isin(championship_ids)]
filtered_captain_matches_df = filtered_captain_matches_df.drop_duplicates(subset=['match_id'], keep='first')
# filtered_captain_matches_df['format'] = np.where(
#     filtered_captain_matches_df['competition_id'].isin(championship_ids),
#     'bo1',
#     'bo3'
# )
filtered_captain_matches_df['winner_id'] = np.where(
    filtered_captain_matches_df['winner'] == 'faction1',
    filtered_captain_matches_df['faction1_team_id'],
    filtered_captain_matches_df['faction2_team_id']
)
filtered_captain_matches_df['loser_id'] = np.where(
    filtered_captain_matches_df['winner'] == 'faction1',
    filtered_captain_matches_df['faction2_team_id'],
    filtered_captain_matches_df['faction1_team_id']
)

filtered_captain_matches_df = filtered_captain_matches_df.drop(columns={'faction1_team_id','faction2_team_id','winner'})
filtered_captain_matches_df = filtered_captain_matches_df.reset_index(drop=True)

In [ ]:
filtered_captain_matches_df 

## Tables building

### dim_championship table

In [ ]:
championship_details_list = []

for data in all_matches:  # juste itérer sur la liste
    if data is None:
        continue  # skip entries with no data

    row = {
        'competition_id': data.get('competition_id'),
        'competition_name': data.get('competition_name'),
        'competition_type': data.get('competition_type'),
        'region': data.get('region'),
        'started_at': data.get('finished_at'), #on utilise la valeur de finished_at comme jour de match car si le match a été dodge il n'y a pas de start_date
    }
    championship_details_list.append(row)

dim_championships = pd.DataFrame(championship_details_list)

In [ ]:
dim_championships['started_at'] = pd.to_datetime(dim_championships['started_at'], unit='s', utc=True).dt.date


In [ ]:
dim_championships[['organizer','season','region','division','sub_region','state','state2','state3']] = dim_championships['competition_name'].str.split(' ',expand=True)
dim_championships['state'] = dim_championships['state2'] + ' ' + dim_championships['state3']
dim_championships.drop(columns=['state2', 'state3'], inplace=True)

In [ ]:
dim_championships = dim_championships.groupby(["competition_id"]).agg({
    "organizer": "first", 
    "competition_type": "first", 
    'region':'first',
    'sub_region':'first',
    "season": "first",       
    "division": "first",               
    "state": "first",
    "started_at" :"first",
}).reset_index()

In [ ]:
dim_championships

### dim_teams and dim_players tables

In [ ]:
team_details_list = []

for team_ids, team_data in all_teams.items():
    if team_data is None:
        continue  # 👈 Skip entries with no data

    row = {
        'team_id': team_data.get('team_id'),
        'team_nickname': team_data.get('nickname'),
        'team_name': team_data.get('name'),
        'team_avatar': team_data.get('avatar'),
        'team_faceit_url': team_data.get('faceit_url'),
    }
    team_details_list.append(row)

dim_teams = pd.DataFrame(team_details_list)
dim_teams = dim_teams.drop_duplicates(subset=['team_id'], keep='last')
dim_teams['team_faceit_url'] = dim_teams['team_faceit_url'].str.replace('/{lang}','')

In [ ]:
#get lis of teams & players from championship request
players_details_list = []

for match_id, match_data in all_stats.items():
    for round_data in match_data['rounds']:
        for team in round_data['teams']:
            team_id = team.get("team_id")
            for player in team["players"]:
                player_id = player["player_id"]
                nickname = player["nickname"]

                row = {
                    "player_id": player_id,
                    "team_id": team_id,
                    "player_name": nickname
                }

                row.update()
                players_details_list.append(row)

players_matches = pd.DataFrame(players_details_list)
players_matches = players_matches.drop_duplicates(subset=['player_id'], keep='last')


In [ ]:
#enrich with detail of players from team_detail request
players_details_list = []

for team_id, team_data in all_teams.items():
    if team_data is None or 'members' not in team_data:
        continue 

    for member in team_data['members']:
        row = {
            'player_id': member.get('user_id'),
            'player_country': member.get('country'),
            'player_faceit_url': member.get('faceit_url'),
            'player_avatar' : member.get('avatar')
        }
        players_details_list.append(row)

# Create DataFrame
players_team_details = pd.DataFrame(players_details_list)
players_team_details = players_team_details.drop_duplicates(subset=['player_id'], keep='last')

In [ ]:
dim_players = pd.merge(players_matches, players_team_details, on='player_id',how='left')
dim_players['player_faceit_url'] = dim_players['player_faceit_url'].str.replace('/{lang}','')

In [ ]:
dim_players

### dim_matches table

The idea is to have a composite key with the combination of match_id and match_round being unique : 

- match id being the whole match (either its a Bo1, Bo3 or Bo5)
- match_round being the id of the map played (1 if a single map is played and then increase up to 5 for Bo5)

In [ ]:
# dim_matches_champ_query_list = []

# for data in all_matches:  
#     if data is None:
#         continue  

#     row = {
#         'match_id': data.get('match_id'),
#         'competition_id': data.get('competition_id'),
#         'match_day': data.get('finished_at'),
#     }
#     dim_matches_champ_query_list.append(row)

# dim_matches_champ_query = pd.DataFrame(dim_matches_champ_query_list)
# dim_matches_champ_query['match_day'] = pd.to_datetime(dim_matches_champ_query['match_day'], unit='s', utc=True).dt.date


In [ ]:
# len(dim_matches_champ_query)

In [ ]:
dim_match_list = []

for match_id, match_data in all_stats.items():
    for round_data in match_data['rounds']:
        for team in round_data['teams']:

            # Get both teams once per match
            teams = round_data['teams']  # Safe because you've pre-filtered
            team1_id = teams[0]['team_id']
            team2_id = teams[1]['team_id']

            for round_data in match_data['rounds']:
                round_stats = round_data['round_stats']
                winner_id = round_stats["Winner"]
                
                dim_match_list.append({
                    "match_id": match_id,
                    "match_round": round_data["match_round"],
                    "format": f"bo{round_data['best_of']}",
                    "map": round_stats["Map"],
                    "total_rounds_played": round_stats["Rounds"]
                })

dim_matches_matchquery = pd.DataFrame(dim_match_list)

In [ ]:
dim_matches_matchquery

In [ ]:
# dim_matches = pd.merge(dim_matches_champ_query[['competition_id','match_id','match_day']], dim_matches_matchquery, on='match_id')
dim_matches = pd.merge(filtered_captain_matches_df, dim_matches_matchquery, on='match_id',how='left')
dim_matches = dim_matches.drop_duplicates(subset=["match_id", "competition_id", "match_day"])

# define format and match_round (working with bo1 and bo3)
dim_matches["round_count"] = dim_matches.groupby(["match_id", "competition_id"])["match_id"].transform("count")
dim_matches["match_round"] = dim_matches.groupby(["match_id", "competition_id"])["match_day"].rank(method="first").astype(int).astype(str)
dim_matches["format"] = dim_matches["round_count"].apply(lambda x: "bo1" if x == 1 else "bo3")
dim_matches = dim_matches.drop(columns=["round_count"])

In [ ]:
dim_matches

In [ ]:
column_order=['match_id','match_round','competition_id','match_day','format','map','total_rounds_played','winner_id','loser_id']
dim_matches = dim_matches[column_order]
dim_matches = dim_matches.drop_duplicates(subset=['match_id','match_round'], keep='first')
dim_matches = dim_matches.reset_index(drop=True)

In [ ]:
dim_matches

In [ ]:
dupes = dim_matches[dim_matches.duplicated(subset=["match_id", "match_round"], keep=False)]
print(dupes)

In [ ]:
dim_matches.dtypes

### fact_players_stats table 

In [ ]:
player_stats_list = []

for match_id, match_data in all_stats.items():
    for round_data in match_data['rounds']:
        map_name = round_data['round_stats'].get('Map')
        winner_id = round_data['round_stats'].get('Winner')

        for team in round_data['teams']:
            team_id = team.get("team_id")
            team_name = team["team_stats"].get("Team")
            team_win = team["team_stats"].get("Team Win")

            for player in team["players"]:
                player_id = player["player_id"]
                nickname = player["nickname"]
                stats = player["player_stats"]

                row = {
                    "match_id": match_id,
                    "match_round": round_data.get("match_round"),
                    "player_id": player_id,
                    "team_id": team_id,
                }

                row.update(stats)
                player_stats_list.append(row)

fact_players_stats = pd.DataFrame(player_stats_list)


In [ ]:
#enrich with competition_id and match_day
fact_players_stats = pd.merge(fact_players_stats, dim_matches[['match_id','competition_id','match_day']], on=['match_id'])

In [ ]:
#correction d'un cas ou les données d'une game sont en doublons, on garde les lignes qui ont les stats les plus importantes, exemple :https://www.faceit.com/fr/cs2/room/1-e4410b46-a6ac-4037-ad35-d7f91f8bf55f/scoreboard
players_stats_sorted = fact_players_stats.sort_values(['match_id', 'player_id', 'Kills'], ascending=[True, True, False])
fact_players_stats = players_stats_sorted.drop_duplicates(subset=['match_id', 'player_id'], keep='first')

In [ ]:
fact_players_stats = fact_players_stats.drop(columns={'Result','Headshots %','K/R Ratio','K/D Ratio','Match Entry Rate','Match 1v1 Win Rate','Flash Success Rate per Match','Sniper Kill Rate per Round','Utility Usage per Round','Flashes per Round in a Match','Sniper Kill Rate per Match','Utility Damage per Round in a Match','Utility Damage Success Rate per Match','Utility Successes','Enemies Flashed per Round in a Match','Utility Success Rate per Match','Match Entry Success Rate','Match 1v2 Win Rate','Utility Enemies'})

In [ ]:
fact_players_stats = fact_players_stats.rename(columns={
    'Zeus Kills':'zeus_kills',
    'Utility Count':'utility_count',
    'Double Kills':'double_kills',
    'Flash Successes':'flash_successes',
    'Quadro Kills':'quadro_kills',
    'Entry Count':'entry_count',
    'First Kills':'first_kills',
    'Flash Count':'flash_count',
    'Sniper Kills':'sniper_kills',
    'Damage':'damage',
    'Utility Damage':'utility_damage',
    'Assists':'assists',
    '1v1Count':'count_1v1',
    'Enemies Flashed':'enemies_flashed',
    'Clutch Kills':'clutch_kills',
    'Penta Kills':'ace',
    'MVPs':'mvps',
    'Deaths':'deaths',
    'Entry Wins':'entry_wins',
    'Kills':'kills',
    '1v2Wins':'wins_1v2',
    '1v1Wins':'wins_1v1',
    'Pistol Kills':'pistol_kills',
    'Knife Kills':'knife_kills',
    'ADR':'adr',
    '1v2Count':'count_1v2',
    'Triple Kills':'triple_kills',
    'Headshots':'headshots',
})

In [ ]:
column_order=['match_id', 'match_round','player_id', 'competition_id','team_id','zeus_kills',
'utility_count',
'double_kills',
'flash_successes',
'quadro_kills',
'entry_count',
'first_kills',
'flash_count',
'sniper_kills',
'damage',
'utility_damage',
'assists',
'count_1v1',
'enemies_flashed',
'clutch_kills',
'ace',
'mvps',
'deaths',
'entry_wins',
'kills',
'wins_1v2',
'wins_1v1',
'pistol_kills',
'knife_kills',
'adr',
'count_1v2',
'triple_kills',
'headshots', 
'match_day']
fact_players_stats = fact_players_stats[column_order]

In [ ]:
fact_players_stats.columns

In [ ]:
exclude_cols = ['match_id','match_round','player_id','competition_id','team_id','match_day']

for col in fact_players_stats.columns:
    if col not in exclude_cols:
        fact_players_stats[col] = pd.to_numeric(fact_players_stats[col], errors="ignore")


In [ ]:
fact_players_stats.dtypes

In [ ]:
dupes = fact_players_stats[fact_players_stats.duplicated(subset=["match_id", "match_round","player_id"], keep=False)]
print(dupes)

### fact_teams_stats table 

In [ ]:
team_stats_list = []

for match_id, match_data in all_stats.items():
    for round_data in match_data['rounds']:
        total_rounds_played = round_data['round_stats'].get('Rounds')

        for team in round_data['teams']:
            team_id = team.get("team_id")
            team_name = team["team_stats"].get("Team")
            team_win = team["team_stats"].get("Team Win")

            row = {
                "match_id": match_id,
                "match_round": round_data.get("match_round"),
                "team_id": team_id,
                "team_win": team_win,
                "total_rounds_played": total_rounds_played
            }

            row.update(team["team_stats"])
            team_stats_list.append(row)

fact_teams_stats = pd.DataFrame(team_stats_list)


In [67]:
fact_teams_stats.head(2)

,match_id,match_round,team_id,team_win,total_rounds_played,First Half Score,Team,Team Win,Final Score,Team Headshots,Second Half Score,Overtime score
0,1-74860e59-e961-4a1c-868c-55ef7d3215b8,1,30191a9c-ceb2-4da0-a146-ec375782ac45,0,12,1,officers,0,1,3.2,0,0
1,1-74860e59-e961-4a1c-868c-55ef7d3215b8,1,f8ac5861-2bb7-4dd1-8378-afa7a9257920,1,12,11,The Toxics,1,11,4.6,0,0


In [68]:
# Enrich with competition_id and match_day - only merge on match_id because comp_id and match_day doesnt differ depending on match_round
fact_teams_stats = pd.merge(fact_teams_stats, dim_matches[['match_id','competition_id','match_day']], on='match_id')

In [69]:
fact_teams_stats

,match_id,match_round,team_id,team_win,total_rounds_played,First Half Score,Team,Team Win,Final Score,Team Headshots,Second Half Score,Overtime score,competition_id,match_day
0,1-74860e59-e961-4a1c-868c-55ef7d3215b8,1,30191a9c-ceb2-4da0-a146-ec375782ac45,0,12,1,officers,0,1,3.2,0,0,3cd81a7e-ed93-43ba-be08-2a1ee7ad7237,2025-07-30 18:40:44
1,1-74860e59-e961-4a1c-868c-55ef7d3215b8,1,f8ac5861-2bb7-4dd1-8378-afa7a9257920,1,12,11,The Toxics,1,11,4.6,0,0,3cd81a7e-ed93-43ba-be08-2a1ee7ad7237,2025-07-30 18:40:44
2,1-14e4f8cb-ca01-47f8-aee1-19136f852ea4,1,6d406cd9-c6a2-4c0d-aa9d-45dfa5812460,0,23,7,5PellMans,0,10,8.6,3,0,4410aa73-9584-4500-a7ca-1504ed6da707,2025-08-19 18:13:11
3,1-14e4f8cb-ca01-47f8-aee1-19136f852ea4,1,36ef04e5-fdbe-43bc-b204-3c6d771dca4e,1,23,5,WHYNOT,1,13,8,8,0,4410aa73-9584-4500-a7ca-1504ed6da707,2025-08-19 18:13:11
4,1-0b1fb845-4c11-42af-8026-743757ba84ea,1,aa598c46-986e-4f63-b06e-b01099a05c37,1,30,6,Tiim,1,16,9,6,4,f62c4c75-f620-40a2-bef8-0be2cb85d2d3,2025-07-16 18:37:21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10841,1-62748ef6-81d8-4a32-aa68-8bb8675e947f,1,5855e03b-fea6-4be3-92af-30bb3ee97d07,1,28,5,VZ_TEAM,1,16,12,7,4,e61be909-1727-4de3-8201-419d89893401,2025-07-28 18:36:00
10842,1-45de182c-e3e5-43be-937c-7646da923c64,1,82bc825b-7a20-4a2f-b588-ee9026342870,1,16,10,Split ESC,1,13,7.8,3,0,4410aa73-9584-4500-a7ca-1504ed6da707,2025-08-18 19:12:23
10843,1-45de182c-e3e5-43be-937c-7646da923c64,1,257df3ae-f478-43d2-a2f1-9e3ac1aa468b,0,16,2,Erlamand,0,3,2.6,1,0,4410aa73-9584-4500-a7ca-1504ed6da707,2025-08-18 19:12:23
10844,1-27d16e1e-7ec0-4295-8e00-e26adaef8ee4,1,4c065180-97d5-408e-8421-4ff9337e72c5,1,18,10,JUMBO TEAM,1,13,6.4,3,0,bd7957b3-1114-435f-af43-22645a508cc0,2025-07-17 18:09:15


In [72]:
fact_teams_stats = fact_teams_stats.drop(columns={'Team','Team Win','Team Headshots'})
fact_teams_stats = fact_teams_stats.rename(columns={'Final Score':'final_score','Overtime score':'overtime_score','First Half Score':'first_half_score','Second Half Score':'second_half_score'})

In [73]:
fact_teams_stats.head(2)

,match_id,match_round,team_id,team_win,total_rounds_played,first_half_score,final_score,second_half_score,overtime_score,competition_id,match_day
0,1-74860e59-e961-4a1c-868c-55ef7d3215b8,1,30191a9c-ceb2-4da0-a146-ec375782ac45,0,12,1,1,0,0,3cd81a7e-ed93-43ba-be08-2a1ee7ad7237,2025-07-30 18:40:44
1,1-74860e59-e961-4a1c-868c-55ef7d3215b8,1,f8ac5861-2bb7-4dd1-8378-afa7a9257920,1,12,11,11,0,0,3cd81a7e-ed93-43ba-be08-2a1ee7ad7237,2025-07-30 18:40:44


In [74]:
#correction d'un cas ou les données d'une game sont en doublons, on garde les lignes qui ont les stats les plus importantes, exemple :https://www.faceit.com/fr/cs2/room/1-e4410b46-a6ac-4037-ad35-d7f91f8bf55f/scoreboard
team_stats_sorted = fact_teams_stats.sort_values(['match_id', 'team_id', 'final_score'], ascending=[True, True, False])
fact_teams_stats = team_stats_sorted.drop_duplicates(subset=['match_id', 'team_id'], keep='first')

column_order=['match_id','match_round','team_id','competition_id','team_win','total_rounds_played','first_half_score','second_half_score','overtime_score','final_score']
fact_teams_stats = fact_teams_stats[column_order]

In [75]:
fact_teams_stats.columns

Index(['match_id', 'match_round', 'team_id', 'competition_id', 'team_win',
       'total_rounds_played', 'first_half_score', 'second_half_score',
       'overtime_score', 'final_score'],
      dtype='object')

In [76]:
fact_teams_stats.dtypes

match_id               object
match_round            object
team_id                object
competition_id         object
team_win               object
total_rounds_played    object
first_half_score       object
second_half_score      object
overtime_score         object
final_score            object
dtype: object

## Summary of tables & columns

In [ ]:
dim_championships.columns

In [ ]:
dim_teams.columns

In [ ]:
dim_players.columns

In [ ]:
dim_matches.columns

In [ ]:
fact_players_stats.columns

In [ ]:
fact_teams_stats.columns

In [ ]:
# Find IDs in fact that are missing in dim
missing_in_dim = fact_players_stats.loc[~fact_players_stats["player_id"].isin(dim_players["player_id"]), "player_id"].unique()

# Find IDs in dim that are missing in fact
missing_in_fact = dim_players.loc[~dim_players["player_id"].isin(fact_players_stats["player_id"]), "player_id"].unique()

print(f"IDs in fact but not in dim: {missing_in_dim}")
print(f"IDs in dim but not in fact: {missing_in_fact}")

# Optional: get the actual fact rows with missing IDs
fact_missing_rows = fact_players_stats[fact_players_stats["player_id"].isin(missing_in_dim)]


## Data ingestion on PostgreSQL DB

In [ ]:
engine = create_engine(
    f"postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
)

In [78]:
primary_keys_map = {
    "dim_championships": ["competition_id"],
    "dim_teams": ["team_id"],
    "dim_players": ["player_id"],
    "dim_matches": ["match_id", "match_round"],
    "fact_teams_stats": ["match_id", "match_round", "team_id"],
    "fact_players_stats": ["match_id", "match_round", "player_id"]
}

def insert_new_records(df, table_name, pk_cols, chunk_size=5000):
    try:
        pk_cols_str = ', '.join(pk_cols)
        query = f"SELECT {pk_cols_str} FROM {table_name}"
        
        existing_pks_df = pd.read_sql(query, engine)

        # Normalize keys to string to avoid datatype mismatches
        for col in pk_cols:
            df[col] = df[col].astype(str)
            existing_pks_df[col] = existing_pks_df[col].astype(str)

        if len(pk_cols) == 1:
            col = pk_cols[0]
            mask = ~df[col].isin(existing_pks_df[col])
        else:
            df_keys = df[pk_cols].apply(tuple, axis=1)
            existing_keys = existing_pks_df.apply(tuple, axis=1)
            mask = ~df_keys.isin(existing_keys)
        
        new_records_df = df[mask]
        
        if not new_records_df.empty:
            new_records_df.to_sql(
                table_name,
                con=engine,
                if_exists='append',
                index=False,
                method='multi',
                chunksize=chunk_size  # <-- added chunking
            )
            print(f"Inserted {len(new_records_df)} new records into {table_name}")
        else:
            print(f"No new records to insert for {table_name}")
    except SQLAlchemyError as e:
        print(f"Database error while inserting into {table_name}: {e}")
    except Exception as e:
        print(f"Unexpected error while inserting into {table_name}: {e}")

def main():
    try:
        insert_new_records(dim_championships, "dim_championships", primary_keys_map["dim_championships"])
        insert_new_records(dim_teams, "dim_teams", primary_keys_map["dim_teams"])
        insert_new_records(dim_players, "dim_players", primary_keys_map["dim_players"])
        insert_new_records(dim_matches, "dim_matches", primary_keys_map["dim_matches"])
        insert_new_records(fact_teams_stats, "fact_teams_stats", primary_keys_map["fact_teams_stats"])
        insert_new_records(fact_players_stats, "fact_players_stats", primary_keys_map["fact_players_stats"])
    except Exception as e:
        print(f"Error in main loading process: {e}")

if __name__ == "__main__":
    main()


No new records to insert for dim_championships
No new records to insert for dim_teams
No new records to insert for dim_players
No new records to insert for dim_matches
No new records to insert for fact_teams_stats
Inserted 54193 new records into fact_players_stats
